# Đánh giá thuật toán so sánh ảnh sử dụng Wavelet Transform

### Mục tiêu:
- Sử dụng wavelet để trích xuất đặc trưng từ hình ảnh
- So sánh độ tương đồng giữa các cặp hình ảnh
- Đánh giá hiệu suất thuật toán thông qua:
  - Độ chính xác (Accuracy)
  - Độ nhạy (Sensitivity/Recall)
  - Độ đặc biệt (Specificity)
  - Đường cong ROC

In [ ]:
# Import các thư viện xử lý ảnh
import cv2  
import numpy as np  
import pywt  

import matplotlib.pyplot as plt  
import seaborn as sns  # Vẽ heatmap 

# Import thư viện đánh giá metrics
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score

# Import ảnh mẫu và hàm tính khoảng cách
from skimage import data  # Dataset ảnh có sẵn trong scikit-image
from scipy.spatial.distance import euclidean, cosine  # Tính khoảng cách vector

# Tắt cảnh báo để output sạch hơn
import warnings
warnings.filterwarnings('ignore')

## 2. Hàm trích xuất đặc trưng sử dụng Wavelet Transform

**Chức năng chính:**
- `extract_wavelet_features()`: Phân rã ảnh thành các hệ số wavelet và trích xuất đặc trưng
  - Input: Ảnh grayscale, loại wavelet (db1, haar...), số level phân rã
  - Output: Vector đặc trưng gồm mean, std, median, max, min từ các hệ số cA, cH, cV, cD
  - Resize ảnh thành kích thước lũy thừa 2 để phân rã wavelet
  
- `visualize_wavelet_decomposition()`: Hiển thị kết quả phân rã wavelet để kiểm tra trực quan

In [ ]:
def extract_wavelet_features(image, wavelet='db1', level=3):
    """
    Trích xuất đặc trưng từ ảnh bằng DWT
    
    Parameters:
        image: Ảnh đầu vào (grayscale)
        wavelet: Loại wavelet (db1, haar, sym, coif)
        level: Số cấp độ phân rã
    
    Returns:
        Vector đặc trưng từ các hệ số wavelet
    """
    # Chuyển sang grayscale nếu ảnh màu (RGB/BGR)
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Resize ảnh thành kích thước lũy thừa của 2 (yêu cầu của DWT)
    # Ví dụ: 300x400 -> 512x512
    h, w = image.shape
    new_h = 2 ** int(np.ceil(np.log2(h)))  # Làm tròn lên lũy thừa 2 gần nhất
    new_w = 2 ** int(np.ceil(np.log2(w)))
    image_resized = cv2.resize(image, (new_w, new_h))
    
    # Phân rã wavelet 2D (Discrete Wavelet Transform)
    # coeffs[0] = cA (approximation), coeffs[1:] = (cH, cV, cD) cho mỗi level
    coeffs = pywt.wavedec2(image_resized, wavelet=wavelet, level=level)
    
    features = []
    
    # === Bước 1: Trích xuất đặc trưng từ hệ số xấp xỉ (cA) ===
    # cA chứa thông tin tần số thấp (nội dung chính của ảnh)
    cA = coeffs[0]
    features.extend([
        np.mean(cA),      # Giá trị trung bình - độ sáng tổng thể
        np.std(cA),       # Độ lệch chuẩn - độ tương phản
        np.median(cA),    # Giá trị trung vị - robust với outliers
        np.max(cA),       # Giá trị sáng nhất
        np.min(cA)        # Giá trị tối nhất
    ])
    
    # === Bước 2: Trích xuất đặc trưng từ các hệ số chi tiết (cH, cV, cD) ===
    # cH = Horizontal details (biên ngang)
    # cV = Vertical details (biên dọc)  
    # cD = Diagonal details (biên chéo)
    for i in range(1, len(coeffs)):
        cH, cV, cD = coeffs[i]  # Hệ số chi tiết ở level i
        
        # Tính đặc trưng cho từng loại chi tiết
        for coeff in [cH, cV, cD]:
            features.extend([
                np.mean(np.abs(coeff)),  # Mean của giá trị tuyệt đối - cường độ biên
                np.std(coeff),           # Std - độ biến thiên của biên
                np.sum(coeff ** 2)       # Energy - tổng năng lượng tín hiệu
            ])
    
    return np.array(features)


def visualize_wavelet_decomposition(image, wavelet='db1', level=2):
    """Hiển thị phân rã wavelet của ảnh"""
    # Chuyển sang grayscale nếu cần
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Resize về lũy thừa 2
    h, w = image.shape
    new_h = 2 ** int(np.ceil(np.log2(h)))
    new_w = 2 ** int(np.ceil(np.log2(w)))
    image_resized = cv2.resize(image, (new_w, new_h))
    
    # Phân rã wavelet
    coeffs = pywt.wavedec2(image_resized, wavelet=wavelet, level=level)
    
    # Chuyển coefficients thành mảng 2D để visualize
    arr, slices = pywt.coeffs_to_array(coeffs)
    
    # Vẽ kết quả
    plt.figure(figsize=(10, 8))
    plt.imshow(np.abs(arr), cmap='gray')  # abs() để hiển thị magnitude
    plt.title(f'Wavelet Decomposition (Level {level}, {wavelet})')
    plt.colorbar()
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    return coeffs

## 3. Hàm tính độ tương đồng giữa 2 ảnh

**Chức năng chính:**
- `calculate_similarity()`: So sánh 2 ảnh dựa trên đặc trưng wavelet
  - Trích xuất đặc trưng wavelet từ cả 2 ảnh
  - Tính khoảng cách giữa 2 vector đặc trưng (Euclidean, Cosine, Correlation)
  - Output: Giá trị khoảng cách (càng nhỏ = càng giống nhau)
  
- `similarity_to_probability()`: Chuyển đổi khoảng cách thành xác suất tương đồng
  - Sử dụng hàm sigmoid để normalize về khoảng [0, 1]
  - Giá trị càng gần 1 = càng giống nhau

In [ ]:
def calculate_similarity(image1, image2, wavelet='db1', level=3, method='euclidean'):
    """
    Tính độ tương đồng giữa 2 ảnh dựa trên đặc trưng wavelet
    
    Parameters:
        image1, image2: Hai ảnh cần so sánh
        wavelet: Loại wavelet
        level: Cấp độ phân rã
        method: Phương pháp tính khoảng cách ('euclidean', 'cosine', 'correlation')
    
    Returns:
        Khoảng cách (càng nhỏ càng giống nhau)
    """
    # Trích xuất đặc trưng wavelet từ cả 2 ảnh
    features1 = extract_wavelet_features(image1, wavelet, level)
    features2 = extract_wavelet_features(image2, wavelet, level)
    
    # Tính khoảng cách giữa 2 vector đặc trưng
    if method == 'euclidean':
        # Euclidean: sqrt(sum((x1-x2)^2)) - khoảng cách hình học
        distance = euclidean(features1, features2)
    elif method == 'cosine':
        # Cosine: 1 - cos(θ) - đo góc giữa 2 vector (0=giống, 1=khác)
        distance = cosine(features1, features2)
    elif method == 'correlation':
        # Correlation: 1 - r - đo mối tương quan tuyến tính
        distance = 1 - np.corrcoef(features1, features2)[0, 1]
    else:
        # Mặc định dùng Euclidean
        distance = euclidean(features1, features2)
    
    return distance


def similarity_to_probability(distance, threshold=100):
    """
    Chuyển khoảng cách thành xác suất tương đồng
    Sử dụng hàm sigmoid: P = 1 / (1 + exp(distance / threshold))
    
    Parameters:
        distance: Khoảng cách giữa 2 ảnh
        threshold: Tham số điều chỉnh độ dốc của sigmoid (càng lớn càng mềm)
    
    Returns:
        Xác suất tương đồng trong khoảng [0, 1]
    """
    # Sigmoid function: chuyển từ (-∞, +∞) về (0, 1)
    # distance = 0 -> probability ≈ 1 (rất giống)
    # distance lớn -> probability ≈ 0 (rất khác)
    probability = 1 / (1 + np.exp(distance / threshold))
    return probability

## 4. Tạo dữ liệu test (các cặp ảnh)

**Chức năng:** Tạo dataset test gồm các cặp ảnh có nhãn để đánh giá thuật toán

**Dataset bao gồm:**
- **Cặp TƯƠNG TỰ (Label = 1):** Ảnh gốc vs các biến thể (nhiễu, xoay, scale, độ sáng)
  - Ví dụ: Camera vs Camera nhiễu, Camera vs Camera xoay 90°
  
- **Cặp KHÁC NHAU (Label = 0):** Các ảnh hoàn toàn khác nội dung
  - Ví dụ: Camera vs Astronaut, Coins vs Moon

**Output:** 
- `image_pairs`: Danh sách các cặp ảnh cần so sánh
- `true_labels`: Nhãn thực tế (1=tương tự, 0=khác nhau)

In [ ]:
def create_test_dataset():
    """Tạo dataset test gồm các cặp ảnh với nhãn (1=tương tự, 0=khác nhau)"""
    # === Load ảnh mẫu từ scikit-image ===
    img1 = data.camera()     # 512x512 grayscale - người với camera
    img2 = data.astronaut()  # 512x512 RGB - phi hành gia
    img3 = data.coins()      # 303x384 grayscale - đồng xu
    img4 = data.moon()       # 512x512 grayscale - mặt trăng
    img5 = data.text()       # 172x448 grayscale - văn bản
    
    # === Tạo các biến thể của ảnh (để test độ robust) ===
    
    # Biến thể của img1 (Camera)
    img1_noisy = (img1 + np.random.normal(0, 10, img1.shape)).astype(np.uint8)  # Thêm nhiễu Gaussian (mean=0, std=10)
    img1_rotated = np.rot90(img1)  # Xoay 90 độ
    img1_scaled = cv2.resize(cv2.resize(img1, (300, 300)), (512, 512))  # Scale xuống rồi lên (mất chất lượng)
    img1_bright = np.clip(img1 * 1.3, 0, 255).astype(np.uint8)  # Tăng độ sáng 30% (clip để không vượt 255)
    
    # Biến thể của img2 (Astronaut)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)  # Chuyển RGB sang grayscale
    img2_noisy = (img2_gray + np.random.normal(0, 5, img2_gray.shape)).astype(np.uint8)  # Nhiễu nhẹ hơn (std=5)
    img2_rotated = np.rot90(img2_gray)  # Xoay 90 độ
    
    # === Các cặp TƯƠNG TỰ (Label = 1) ===
    # Ảnh gốc vs các biến thể - test khả năng nhận diện qua nhiễu/biến đổi
    similar_pairs = [
        (img1, img1_noisy, "Camera - Camera nhiễu"),
        (img1, img1_rotated, "Camera - Camera xoay"),
        (img1, img1_scaled, "Camera - Camera scale"),
        (img1, img1_bright, "Camera - Camera sáng"),
        (img2_gray, img2_noisy, "Astronaut - Astronaut nhiễu"),
        (img2_gray, img2_rotated, "Astronaut - Astronaut xoay"),
    ]
    
    # === Các cặp KHÁC NHAU (Label = 0) ===
    # Các ảnh hoàn toàn khác nội dung - test khả năng phân biệt
    dissimilar_pairs = [
        (img1, img2_gray, "Camera - Astronaut"),
        (img1, img3, "Camera - Coins"),
        (img1, img4, "Camera - Moon"),
        (img1, img5, "Camera - Text"),
        (img2_gray, img3, "Astronaut - Coins"),
        (img2_gray, img4, "Astronaut - Moon"),
        (img3, img4, "Coins - Moon"),
        (img3, img5, "Coins - Text"),
        (img4, img5, "Moon - Text"),
    ]
    
    # === Tổng hợp dataset ===
    image_pairs = []
    labels = []
    
    # Thêm các cặp tương tự (label = 1)
    for pair in similar_pairs:
        image_pairs.append((pair[0], pair[1], pair[2]))
        labels.append(1)
    
    # Thêm các cặp khác nhau (label = 0)
    for pair in dissimilar_pairs:
        image_pairs.append((pair[0], pair[1], pair[2]))
        labels.append(0)
    
    return image_pairs, np.array(labels)


# === Tạo dataset và in thông tin ===
image_pairs, true_labels = create_test_dataset()
print(f"Tổng số cặp ảnh: {len(image_pairs)}")
print(f"Số cặp tương tự: {np.sum(true_labels == 1)}")
print(f"Số cặp khác nhau: {np.sum(true_labels == 0)}")

## 5. Hiển thị một số cặp ảnh mẫu

**Chức năng:** Visualize 6 cặp ảnh đầu tiên trong dataset để kiểm tra trực quan

**Mục đích:**
- Xem các cặp ảnh tương tự và khác nhau trông như thế nào
- Kiểm tra tính hợp lý của dataset
- Hiển thị màu xanh cho cặp tương tự, đỏ cho cặp khác nhau
- Lưu kết quả vào thư mục `results/`

In [ ]:
# Hiển thị một số cặp ảnh
fig, axes = plt.subplots(3, 4, figsize=(16, 12))  # 3 hàng x 4 cột (6 cặp ảnh, mỗi cặp 2 ảnh)

# Duyệt qua 6 cặp ảnh đầu tiên
for idx in range(6):
    img1, img2, desc = image_pairs[idx]
    label = "TƯƠNG TỰ" if true_labels[idx] == 1 else "KHÔNG TƯƠNG TỰ"
    
    # Tính vị trí trong grid 3x4
    row = idx // 2          # Hàng (0, 1, 2): 0,1->0; 2,3->1; 4,5->2
    col = (idx % 2) * 2     # Cột (0, 2): cặp lẻ->0, cặp chẵn->2
    
    # Ảnh 1 (cột bên trái)
    axes[row, col].imshow(img1, cmap='gray')
    axes[row, col].set_title(f'Ảnh 1', fontsize=10)
    axes[row, col].axis('off')
    
    # Ảnh 2 (cột bên phải)
    axes[row, col+1].imshow(img2, cmap='gray')
    axes[row, col+1].set_title(f'Ảnh 2\n{desc}\n[{label}]', fontsize=10, 
                                color='green' if label == "TƯƠNG TỰ" else 'red')  # Màu theo nhãn
    axes[row, col+1].axis('off')

plt.tight_layout()
plt.savefig('results/sample_pairs.png', dpi=150, bbox_inches='tight')  # Lưu với độ phân giải cao
plt.show()

## 6. Tính toán điểm tương đồng cho tất cả các cặp

**Chức năng:** Tính khoảng cách và xác suất tương đồng cho mọi cặp ảnh trong dataset

**Quy trình:**
1. Duyệt qua từng cặp ảnh
2. Tính khoảng cách Euclidean giữa vector đặc trưng wavelet
3. Chuyển đổi khoảng cách thành xác suất (0-1)
4. Lưu kết quả để đánh giá

**Output:**
- `distances`: Mảng chứa khoảng cách cho mọi cặp
- `probabilities`: Mảng chứa xác suất tương đồng

In [ ]:
# Tính khoảng cách cho tất cả các cặp
distances = []
probabilities = []

print("Đang tính toán độ tương đồng...")
for idx, (img1, img2, desc) in enumerate(image_pairs):
    # Tính khoảng cách Euclidean dựa trên đặc trưng wavelet
    # wavelet='db1': Daubechies 1 (Haar wavelet)
    # level=3: Phân rã 3 cấp độ
    # method='euclidean': Khoảng cách Euclidean
    distance = calculate_similarity(img1, img2, wavelet='db1', level=3, method='euclidean')
    
    # Chuyển khoảng cách thành xác suất tương đồng [0, 1]
    # threshold=50: Điều chỉnh độ nhạy của sigmoid
    prob = similarity_to_probability(distance, threshold=50)
    
    distances.append(distance)
    probabilities.append(prob)
    
    # In kết quả từng cặp
    print(f"Cặp {idx+1}/{len(image_pairs)}: {desc[:30]:30s} | Distance: {distance:8.2f} | Prob: {prob:.4f} | Label: {true_labels[idx]}")

# Chuyển sang numpy array để xử lý tiếp
distances = np.array(distances)
probabilities = np.array(probabilities)

print("\n✓ Hoàn thành tính toán độ tương đồng!")

## 7. Tính toán các metrics đánh giá

**Chức năng:** Đánh giá hiệu suất thuật toán tại ngưỡng tối ưu

**Quy trình:**
1. Test nhiều ngưỡng khác nhau (threshold)
2. Tại mỗi ngưỡng: distance < threshold → Dự đoán "Tương tự" (1), ngược lại → "Khác nhau" (0)
3. Tính các metrics: Accuracy, Sensitivity, Specificity, Precision
4. Tìm ngưỡng cho accuracy cao nhất

**Các metrics:**
- **Accuracy**: Tỷ lệ dự đoán đúng tổng thể
- **Sensitivity (Recall/TPR)**: Khả năng phát hiện các cặp tương tự
- **Specificity (TNR)**: Khả năng phát hiện các cặp khác nhau
- **Precision**: Độ chính xác khi dự đoán "tương tự"

In [ ]:
def calculate_metrics_at_threshold(distances, true_labels, threshold):
    """
    Tính metrics tại một ngưỡng cụ thể
    distance < threshold => Dự đoán = 1 (Tương tự)
    distance >= threshold => Dự đoán = 0 (Khác nhau)
    """
    # Phân loại dựa trên ngưỡng
    predictions = (distances < threshold).astype(int)
    
    # === Tính Confusion Matrix ===
    # tn: True Negative - dự đoán đúng cặp khác nhau
    # fp: False Positive - dự đoán sai là tương tự (Type I error)
    # fn: False Negative - dự đoán sai là khác nhau (Type II error)
    # tp: True Positive - dự đoán đúng cặp tương tự
    tn, fp, fn, tp = confusion_matrix(true_labels, predictions).ravel()
    
    # === Tính các metrics ===
    # Accuracy: Tỷ lệ dự đoán đúng = (TP + TN) / Tổng số
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    
    # Sensitivity (Recall/TPR): Tỷ lệ phát hiện đúng cặp tương tự = TP / (TP + FN)
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    # Specificity (TNR): Tỷ lệ phát hiện đúng cặp khác nhau = TN / (TN + FP)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    # Precision (PPV): Độ chính xác khi dự đoán tương tự = TP / (TP + FP)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    
    return {
        'threshold': threshold,
        'accuracy': accuracy,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'precision': precision,
        'TP': tp, 'TN': tn, 'FP': fp, 'FN': fn,
        'predictions': predictions
    }


# === Tìm ngưỡng tối ưu (maximize accuracy) ===
# Test 50 ngưỡng khác nhau trong khoảng [min_distance, max_distance]
thresholds_to_test = np.linspace(distances.min(), distances.max(), 50)
best_metrics = None
best_accuracy = 0

# Duyệt qua từng ngưỡng để tìm ngưỡng cho accuracy cao nhất
for thresh in thresholds_to_test:
    metrics = calculate_metrics_at_threshold(distances, true_labels, thresh)
    if metrics['accuracy'] > best_accuracy:
        best_accuracy = metrics['accuracy']
        best_metrics = metrics

# === In kết quả đánh giá ===
print("=" * 70)
print("KẾT QUẢ ĐÁNH GIÁ TẠI NGƯỠNG TỐI ƯU")
print("=" * 70)
print(f"\nNgưỡng tối ưu: {best_metrics['threshold']:.2f}")
print(f"\n{'Metric':<20} {'Giá trị':<15} {'Ý nghĩa'}")
print("-" * 70)
print(f"{'Accuracy':<20} {best_metrics['accuracy']:.4f} ({best_metrics['accuracy']*100:.2f}%)     Tỷ lệ phân loại đúng")
print(f"{'Sensitivity':<20} {best_metrics['sensitivity']:.4f} ({best_metrics['sensitivity']*100:.2f}%)     Phát hiện cặp tương tự")
print(f"{'Specificity':<20} {best_metrics['specificity']:.4f} ({best_metrics['specificity']*100:.2f}%)     Phát hiện cặp khác nhau")
print(f"{'Precision':<20} {best_metrics['precision']:.4f} ({best_metrics['precision']*100:.2f}%)     Độ chính xác dự đoán")
print("\nConfusion Matrix:")
print(f"  True Positive (TP):  {best_metrics['TP']:3d} - Tương tự → Tương tự")
print(f"  True Negative (TN):  {best_metrics['TN']:3d} - Khác nhau → Khác nhau")
print(f"  False Positive (FP): {best_metrics['FP']:3d} - Khác nhau → Tương tự (sai)")
print(f"  False Negative (FN): {best_metrics['FN']:3d} - Tương tự → Khác nhau (sai)")
print("=" * 70)

## 8. Vẽ Confusion Matrix

**Chức năng:** Visualize ma trận nhầm lẫn (Confusion Matrix) để đánh giá chi tiết

**Confusion Matrix bao gồm:**
- **True Positive (TP)**: Dự đoán đúng cặp tương tự
- **True Negative (TN)**: Dự đoán đúng cặp khác nhau
- **False Positive (FP)**: Dự đoán sai - cho là tương tự nhưng thực tế khác nhau
- **False Negative (FN)**: Dự đoán sai - cho là khác nhau nhưng thực tế tương tự

**Mục đích:** Hiểu rõ thuật toán đang sai ở đâu và bao nhiêu trường hợp

In [ ]:
# Vẽ Confusion Matrix
cm = confusion_matrix(true_labels, best_metrics['predictions'])

plt.figure(figsize=(8, 6))

# Vẽ heatmap với seaborn
# annot=True: Hiển thị số trong ô
# fmt='d': Format số nguyên
# cmap='Blues': Màu xanh dương gradient
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Không tương tự', 'Tương tự'],
            yticklabels=['Không tương tự', 'Tương tự'],
            cbar_kws={'label': 'Số lượng'})

plt.xlabel('Dự đoán', fontsize=12, weight='bold')
plt.ylabel('Thực tế', fontsize=12, weight='bold')
plt.title(f'Confusion Matrix\n(Threshold = {best_metrics["threshold"]:.2f}, Accuracy = {best_metrics["accuracy"]:.2%})', 
          fontsize=14, weight='bold')

# Thêm annotations với font lớn hơn
for i in range(2):
    for j in range(2):
        plt.text(j + 0.5, i + 0.7, 
                f'{cm[i, j]}', 
                ha='center', va='center', 
                fontsize=20, weight='bold', 
                color='white' if cm[i, j] > cm.max()/2 else 'black')  # Màu chữ adaptive

plt.tight_layout()
plt.savefig('results/confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Vẽ đường cong ROC (Receiver Operating Characteristic)

**Chức năng:** Vẽ đường cong ROC để đánh giá hiệu suất tổng thể của thuật toán

**ROC Curve:**
- Trục X: False Positive Rate (FPR) = 1 - Specificity
- Trục Y: True Positive Rate (TPR) = Sensitivity
- Đường cong thể hiện trade-off giữa TPR và FPR ở các ngưỡng khác nhau

**AUC (Area Under Curve):**
- AUC = 1.0: Phân loại hoàn hảo
- AUC = 0.5: Phân loại ngẫu nhiên (như tung đồng xu)
- AUC càng cao càng tốt (>0.8 là tốt, >0.9 là xuất sắc)

**Điểm tối ưu:** Điểm có Youden's Index (TPR - FPR) lớn nhất

In [ ]:
# === Tính ROC curve ===
# Sử dụng probabilities (xác suất) thay vì distances
# fpr: False Positive Rate (1 - Specificity)
# tpr: True Positive Rate (Sensitivity)
# thresholds_roc: Các ngưỡng xác suất tương ứng
fpr, tpr, thresholds_roc = roc_curve(true_labels, probabilities)
roc_auc = auc(fpr, tpr)  # Tính diện tích dưới đường cong ROC

# === Tìm điểm tối ưu (Youden's Index) ===
# Youden's Index = TPR - FPR (maximize)
# Điểm cân bằng tốt nhất giữa Sensitivity và Specificity
youden_index = tpr - fpr
optimal_idx = np.argmax(youden_index)
optimal_threshold_roc = thresholds_roc[optimal_idx]  # Ngưỡng xác suất tối ưu
optimal_fpr = fpr[optimal_idx]  # FPR tại điểm tối ưu
optimal_tpr = tpr[optimal_idx]  # TPR tại điểm tối ưu

# === Vẽ ROC curve ===
plt.figure(figsize=(10, 8))

# Đường ROC chính
plt.plot(fpr, tpr, color='darkorange', lw=2, 
         label=f'ROC curve (AUC = {roc_auc:.3f})')

# Đường tham chiếu (random classifier)
# Classifier ngẫu nhiên có AUC = 0.5
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', 
         label='Random Classifier (AUC = 0.500)')

# Đánh dấu điểm tối ưu
plt.plot(optimal_fpr, optimal_tpr, 'ro', markersize=10, 
         label=f'Optimal Point (TPR={optimal_tpr:.3f}, FPR={optimal_fpr:.3f})')

# === Cấu hình biểu đồ ===
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - Specificity)', fontsize=12, weight='bold')
plt.ylabel('True Positive Rate (Sensitivity)', fontsize=12, weight='bold')
plt.title('Receiver Operating Characteristic (ROC) Curve\nWavelet-based Image Similarity', 
          fontsize=14, weight='bold')
plt.legend(loc="lower right", fontsize=11)
plt.grid(alpha=0.3)

# Thêm text box với thông tin chi tiết
textstr = f'AUC = {roc_auc:.4f}\nOptimal Threshold = {optimal_threshold_roc:.4f}\nTPR = {optimal_tpr:.4f}\nFPR = {optimal_fpr:.4f}'
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(0.6, 0.2, textstr, fontsize=10, verticalalignment='top', bbox=props)

plt.tight_layout()
plt.savefig('results/roc_curve.png', dpi=150, bbox_inches='tight')
plt.show()

# === Giải thích AUC ===
print(f"\n{'='*70}")
print(f"AUC (Area Under Curve): {roc_auc:.4f}")
print(f"{'='*70}")
print(f"\nĐánh giá AUC:")
print(f"  • 0.9 - 1.0: Xuất sắc")
print(f"  • 0.8 - 0.9: Tốt")
print(f"  • 0.7 - 0.8: Khá")
print(f"  • 0.5 - 0.7: Trung bình")
print(f"  • < 0.5: Kém")

## 10. Phân tích chi tiết hiệu suất ở các ngưỡng khác nhau

**Chức năng:** Vẽ biểu đồ thể hiện sự thay đổi của các metrics theo ngưỡng

**Mục đích:**
- Hiểu cách các metrics thay đổi khi ta điều chỉnh ngưỡng phân loại
- Tìm ngưỡng cân bằng giữa Sensitivity và Specificity
- Quan sát trade-off giữa các metrics khác nhau

**4 biểu đồ:**
1. Accuracy vs Threshold
2. Sensitivity vs Threshold  
3. Specificity vs Threshold
4. Precision vs Threshold

Đường đỏ dọc chỉ ngưỡng tối ưu đã tìm được ở bước 7

In [ ]:
# === Tính metrics cho nhiều ngưỡng ===
# Test 100 ngưỡng khác nhau để có đồ thị mượt mà
test_thresholds = np.linspace(distances.min(), distances.max(), 100)
metrics_list = [calculate_metrics_at_threshold(distances, true_labels, t) for t in test_thresholds]

# Trích xuất các giá trị để vẽ
# Format: metric_name: (values_list, color, optimal_value)
metrics_data = {
    'Accuracy': ([m['accuracy'] for m in metrics_list], 'b-', best_metrics['accuracy']),
    'Sensitivity': ([m['sensitivity'] for m in metrics_list], 'g-', best_metrics['sensitivity']),
    'Specificity': ([m['specificity'] for m in metrics_list], 'orange', best_metrics['specificity']),
    'Precision': ([m['precision'] for m in metrics_list], 'purple', best_metrics['precision'])
}

# === Vẽ 4 biểu đồ trong 1 figure ===
fig, axes = plt.subplots(2, 2, figsize=(15, 10))  # Grid 2x2
axes = axes.flatten()  # Chuyển từ 2D array thành 1D để dễ index

# Vẽ từng metric
for idx, (metric_name, (values, color, optimal_value)) in enumerate(metrics_data.items()):
    ax = axes[idx]
    
    # Vẽ đường chính (metric vs threshold)
    ax.plot(test_thresholds, values, color, linewidth=2, label=metric_name)
    
    # Đường thẳng đứng: ngưỡng tối ưu từ bước 7
    ax.axvline(best_metrics['threshold'], color='r', linestyle='--', 
               label=f'Optimal Threshold = {best_metrics["threshold"]:.2f}')
    
    # Đường ngang: giá trị tối ưu của metric
    ax.axhline(optimal_value, color='gray', linestyle=':', alpha=0.5)
    
    # Cấu hình trục và tiêu đề
    ax.set_xlabel('Threshold', fontsize=11)
    ax.set_ylabel(metric_name, fontsize=11)
    ax.set_title(f'{metric_name} vs Threshold', fontsize=12, weight='bold')
    ax.grid(alpha=0.3)
    ax.legend()

plt.tight_layout()
plt.savefig('results/metrics_vs_threshold.png', dpi=150, bbox_inches='tight')
plt.show()

## 11. Hiển thị ví dụ về wavelet decomposition

**Chức năng:** Visualize kết quả phân rã wavelet của một ảnh mẫu

**Mục đích:**
- Hiểu trực quan cách wavelet phân rã ảnh thành các thành phần tần số khác nhau
- Kiểm tra các hệ số wavelet (cA, cH, cV, cD) được tạo ra
- Verify rằng quá trình phân rã wavelet hoạt động đúng

**Output:** Hình ảnh hiển thị các hệ số wavelet ở nhiều level khác nhau

In [ ]:
# === Hiển thị wavelet decomposition cho ảnh mẫu ===
sample_image = data.camera()  # Ảnh Camera 512x512
print("Wavelet Decomposition:")

# Phân rã và visualize
# wavelet='db1': Daubechies 1 (Haar)
# level=3: 3 cấp độ phân rã (càng nhiều level càng chi tiết)
visualize_wavelet_decomposition(sample_image, wavelet='db1', level=3)